In [1]:
import sys
import importlib

from langchain.chat_models.openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.tools import Tool

# quickfix, since modules from relative imports were not found
sys.path.append("/Users/stefanie/Documents/code/RAG_Langchain")
importlib.import_module("modules.prompts", package="RAG_Langchain")
importlib.import_module("modules.agent", package="RAG_Langchain")

from modules.prompts import DebugPrompt, MobyPrompt
from modules.retriever import Retriever

In [2]:
# define user query
query = "Find a sentence that would be terrible advice for a DIY project."

In [3]:
# get additional context from embedding retriever
retriever = Retriever().retriever
#retriever = retriever.embed().retrieve()

In [4]:
# get prompt
prompt = MobyPrompt.prompt
#prompt = DebugPrompt.prompt

In [5]:
# define llm model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_stop = llm.bind(stop=["\nQuestion:"])

In [6]:
# legacy way
llmchain = LLMChain(llm=llm_with_stop, prompt=prompt, output_key='output', verbose=True)
resp = llmchain({'context': retriever.context, 'query': query})
print(resp['output'])

AttributeError: 'VectorStoreRetriever' object has no attribute 'context'

In [9]:
# LCEL I
chain = ({"context": RunnablePassthrough(), "query": RunnablePassthrough()}) | prompt | llm | StrOutputParser()

chain.invoke({"context": retriever.context,
              "query": query})


AttributeError: 'VectorStoreRetriever' object has no attribute 'context'

In [8]:
# LCEL II
chain = ({"context": retriever.retriever, "query": RunnablePassthrough()}) | prompt | llm | StrOutputParser()

chain.invoke(query)

AttributeError: 'VectorStoreRetriever' object has no attribute 'retriever'

In [ ]:
# define tools
tools = [
    Tool.from_function(
        func=retriever.retrieve,
        name="DocAgent",
        description="useful for when you need to search for answer from documents",
    ),
]

In [ ]:
# define langchain agent
from langchain.agents import initialize_agent, AgentType
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         agent_scratchpad="intermediate_steps",
                         verbose=True)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [ ]:
# add memory
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)

In [ ]:
# define agent executer
from langchain.agents.agent import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": query})['output']

In [ ]:
# run testsuite
pass